# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [137]:
# imports
import plotly.express as px
import panel as pn
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
#pn.extension('plotly')
import hvplot.pandas
pn.extension()
from matplotlib.figure import Figure

In [138]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")

# Set token using Plotly Express set function
px.set_mapbox_access_token(mapbox_token)

# Import Data

In [139]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [140]:
# Set token using Plotly Express set function
px.set_mapbox_access_token(mapbox_token)
    
# Build housing units df
housing_units = sfo_data.groupby('year')['housing_units'].mean()
    
housing_units_year = housing_units.to_frame()
housing_units_year.reset_index(inplace=True)

# Get slice of dataframe of only year, gross rent and sale price per square foot
grossrent_salesprice = sfo_data.loc[:, ['sale_price_sqr_foot', 'gross_rent']]

# Calculate average gross rent and sales price for dataframe grossrent_salesprice
avg_grossrent_salesprice = grossrent_salesprice.groupby('year').mean()
avg_grossrent_salesprice.reset_index(inplace=True)

neighborhood_prices = sfo_data.copy()
avg_neighborhood_prices = neighborhood_prices.groupby(['year', 'neighborhood']).mean()
avg_neighborhood_prices.reset_index(inplace=True)
 
# Getting the data from the top 10 expensive neighborhoods
neighborhood_all_years_prices = neighborhood_prices.loc[:, ['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent']]
avg_neighborhood_all_years_prices = neighborhood_all_years_prices.groupby('neighborhood').mean()

top_10_expensive_neighborhood = avg_neighborhood_all_years_prices.nlargest(10, 'sale_price_sqr_foot')
avg_neighborhood_all_years_prices.reset_index(inplace=True)

top_10_expensive_neighborhood.reset_index(inplace=True)

# Calculate the mean values for each neighborhood
avg_neighborhood_all_years_prices.rename(columns={'neighborhood':'Neighborhood'}, inplace=True)
#avg_neighborhood_all_years_prices.head()

# Join the average values with the neighborhood locations
neighborhood_sales_loc = pd.merge(df_neighborhood_locations, avg_neighborhood_all_years_prices, 
                                  on='Neighborhood',
                                  #axis=1, 
                                  #join='inner',
                                  )

In [141]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    housing_units_per_year_plot = housing_units_year.hvplot.bar(
        x="year", 
        y="housing_units", 
        rot=90, 
        xlabel="Year",
        ylabel="Housing Units",
        ylim=(367500,387500),
        title='Housing Units in San Francisco from 2010 to 2016'
        )
    
    return housing_units_per_year_plot

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
      
    average_gross_rent_plot = avg_grossrent_salesprice.hvplot.line(
                                    x="year", 
                                    y="gross_rent",
                                    xlabel="Year",
                                    ylabel="Gross Rent",
                                    title="Average Gross Rent in San Francisco"
                                    )

    return average_gross_rent_plot

def average_sales_price():
    """Average Sales Price Per Year."""
    
    average_grossrent_salesprice_plot = avg_grossrent_salesprice.hvplot.line(
                                            x="year", 
                                            y="sale_price_sqr_foot",
                                            xlabel="Year", 
                                            ylabel="Avg. Sale Price",
                                            title="Average Sale Price per Square Foot in San Francisco"
                                            )
    
    return average_grossrent_salesprice_plot

def choose_neighborhood(neighborhood):
   
    # create dataframe [neighborhood_df] with rows of only the selected neighborhood from drop down 
    neighborhood_df = avg_neighborhood_prices[avg_neighborhood_prices["neighborhood"] == neighborhood]

    plot_title = "neighborhood:" + ' ' + neighborhood
    
    # Plot neighborhood dataframe as line plot
    return neighborhood_df.hvplot.line(
            x="year",
            y="sale_price_sqr_foot",
            xlabel="Year",
            ylabel="Avg. Sale Price per Square Foot",
            colormap="viridis",
            title=plot_title,
            frame_height=300,        
            frame_width=600,        
             )

# Define function to choose a neighborhood and return the line plot

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    list_of_neighborhoods = avg_neighborhood_prices.neighborhood.unique().tolist()
    return interact(choose_neighborhood, neighborhood=list_of_neighborhoods)


def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    

    top_10_exp_neighborhoods = top_10_expensive_neighborhood.hvplot.bar(
        x="neighborhood", 
        y="sale_price_sqr_foot", 
        rot=90, 
        title='Top 10 Expensive Neighborhoods in SFO',
        xlabel='Neighborhood', 
        ylabel='Avg. Sale Price per Square Foot',
        frame_height=300,
        frame_width=600,
        )

    return top_10_exp_neighborhoods

def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    # YOUR CODE HERE!
    parallel_coordinates_plot = px.parallel_coordinates(top_10_expensive_neighborhood, color='sale_price_sqr_foot')

    return parallel_coordinates_plot

def parallel_categories():
    """Parallel Categories Plot."""
    
    # YOUR CODE HERE!
    parallel_categories_plot = px.parallel_categories(top_10_expensive_neighborhood,
                                                      color="sale_price_sqr_foot",
                                                      color_continuous_scale=px.colors.sequential.Inferno,
                                                    )

    return parallel_categories_plot

def neighborhood_map():
    """Neighborhood Map"""

    neighborhood_plot = px.scatter_mapbox(
        neighborhood_sales_loc,
        lat="Lat",
        lon="Lon",
        size="sale_price_sqr_foot",
        hover_name="Neighborhood",
        color="gross_rent",
        color_continuous_scale=px.colors.cyclical.IceFire,
        title="Average Sale Price Per Square Foot and Gross Rent in San Francisco",
        zoom=11,
        width=1000,
    )
    return neighborhood_plot

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [142]:
# Create panels to structure the layout of the dashboard
welcome_map_column = pn.Column("## SFO Neighborhood", neighborhood_map())

neighborhood_column = pn.Column("## Housing Units Per Year", 
                                housing_units_per_year(),
                                average_sales_price(),
                                average_gross_rent(),
                                )

neighborhood_analysis_column = pn.Column("## Neighborhood Analysis", 
                                        average_price_by_neighborhood(),
                                        top_most_expensive_neighborhoods(),
                                        )

parallel_plot_analysis_column = pn.Column("## Parallel Plot Analysis",
                                         parallel_coordinates(),
                                         parallel_categories())

# Create tabs
real_estate_dashboard = pn.Tabs(
    ("Welcome", welcome_map_column),
    ("Housing Analysis", neighborhood_column),
    ("Neighborhood Analysis", neighborhood_analysis_column),
    ("Parallel Plots Analysis", parallel_plot_analysis_column),
)

## Serve the Panel Dashboard

In [143]:
real_estate_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [1] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] HoloViews(Curve)
        [3] HoloViews(Curve)
    [2] Column
        [0] Markdown(str)
        [1] Column
            [0] Column
                [0] Select(name='neighborhood', options=['Alamo Square', ...], value='Alamo Square')
            [1] Row
                [0] HoloViews(Curve, name='interactive28565')
        [2] HoloViews(Bars)
    [3] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)